# custom word embeddings

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential

In [3]:
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Flatten


In [4]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [5]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [6]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [7]:
all_words

['This',
 'is',
 'an',
 'excellent',
 'movie',
 'The',
 'move',
 'was',
 'fantastic',
 'I',
 'like',
 'it',
 'You',
 'should',
 'watch',
 'it',
 'is',
 'brilliant',
 'Exceptionally',
 'good',
 'Wonderfully',
 'directed',
 'and',
 'executed',
 'I',
 'like',
 'it',
 'Its',
 'a',
 'fantastic',
 'series',
 'Never',
 'watched',
 'such',
 'a',
 'brillent',
 'movie',
 'It',
 'is',
 'a',
 'Wonderful',
 'movie',
 'horrible',
 'acting',
 'waste',
 'of',
 'money',
 'pathetic',
 'picture',
 'It',
 'was',
 'very',
 'boring',
 'I',
 'did',
 'not',
 'like',
 'the',
 'movie',
 'The',
 'movie',
 'was',
 'horrible',
 'I',
 'will',
 'not',
 'recommend',
 'The',
 'acting',
 'is',
 'pathetic']

In [8]:
unique_words = set(all_words)
print(len(unique_words))

45


In [9]:
# Since hashing is used there is a probability that different words will be hashed to the same index. 
# The probability of a non-unique hash is proportional to the vocabulary size selected. 
# It is suggested by Jason Brownlee Jason Brownlee to use a vocabulary size 25% larger than the
# word size to increase the uniqueness of the hashes.
import math
# from tensorflow.random import set_random_seed
# set_random_seed(1)
import tensorflow
tensorflow.random.set_seed(1)

In [10]:
embedded_sentences = [one_hot(sent, math.ceil(45*1.50)) for sent in corpus]
print(embedded_sentences )

[[48, 27, 62, 18, 52], [23, 51, 18, 2, 34, 63, 57], [8, 6, 52, 57, 27, 48], [58, 24], [66, 20, 38, 34, 34, 63, 57], [40, 17, 2, 6], [1, 28, 54, 17, 14, 52], [57, 27, 17, 22, 52], [42, 55], [1, 21, 8], [66, 57], [57, 18, 36, 50], [34, 39, 10, 63, 23, 52], [23, 52, 18, 42], [34, 50, 10, 43], [23, 55, 27, 66]]


In [11]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [12]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[48 27 62 18 52  0  0]
 [23 51 18  2 34 63 57]
 [ 8  6 52 57 27 48  0]
 [58 24  0  0  0  0  0]
 [66 20 38 34 34 63 57]
 [40 17  2  6  0  0  0]
 [ 1 28 54 17 14 52  0]
 [57 27 17 22 52  0  0]
 [42 55  0  0  0  0  0]
 [ 1 21  8  0  0  0  0]
 [66 57  0  0  0  0  0]
 [57 18 36 50  0  0  0]
 [34 39 10 63 23 52  0]
 [23 52 18 42  0  0  0]
 [34 50 10 43  0  0  0]
 [23 55 27 66  0  0  0]]


In [13]:
model = Sequential()
model.add(Embedding(100, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [14]:
#print(model.layers[0].add_weights())
print(model.layers[0].get_weights())

[array([[-0.03348692,  0.04014813,  0.01309742, ..., -0.02473292,
         0.03862232,  0.03872934],
       [ 0.02872816, -0.0440448 , -0.04289062, ...,  0.02339447,
         0.04197038,  0.01685185],
       [-0.02839044, -0.03134662, -0.00928329, ...,  0.039776  ,
        -0.01640397,  0.04187632],
       ...,
       [-0.01823374,  0.02428428, -0.0360656 , ..., -0.00657698,
        -0.00366609,  0.04443605],
       [-0.03856635, -0.04159343, -0.00864277, ...,  0.0249463 ,
         0.03822659, -0.03449532],
       [-0.0264432 ,  0.00476432, -0.02046658, ..., -0.02385641,
        -0.03841426,  0.01261011]], dtype=float32)]


In [15]:
print(model.layers[0].add_weight())

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.1406504>


In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())
#the output shape of embedding layer is (batch_size, input_length, output_dim),
#input_length argument (length of input sequences)
#model.add(Embedding(input_dim,output_dim, input_length=length_long_sentence))
#model.add(Embedding(50, 20, input_length=length_long_sentence))
#length_long_sentence: 7

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 20)             2001      
_________________________________________________________________
flatten (Flatten)            (None, 140)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 141       
Total params: 2,142
Trainable params: 2,142
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
import keras.backend as K
#learning rate
print(K.eval(model.optimizer.lr))

0.001


In [18]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6877 - acc: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6839 - acc: 0.6875
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6802 - acc: 0.8750
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6764 - acc: 0.9375
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6727 - acc: 0.9375
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6689 - acc: 0.9375
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6652 - acc: 0.9375
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6615 - acc: 0.9375
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6577 - acc: 0.9375
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6540 - acc: 0.9375
Epoch 11/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6503 

1/1 [==============================] - 0s 2ms/step - loss: 0.3168 - acc: 1.0000
Epoch 85/100
1/1 [==============================] - 0s 7ms/step - loss: 0.3123 - acc: 1.0000
Epoch 86/100
1/1 [==============================] - 0s 3ms/step - loss: 0.3079 - acc: 1.0000
Epoch 87/100
1/1 [==============================] - 0s 3ms/step - loss: 0.3035 - acc: 1.0000
Epoch 88/100
1/1 [==============================] - 0s 3ms/step - loss: 0.2991 - acc: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 3ms/step - loss: 0.2948 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - ETA: 0s - loss: 0.2905 - acc: 1.000 - 0s 3ms/step - loss: 0.2905 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2862 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 7ms/step - loss: 0.2820 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 3ms/step - loss: 0.2779 - acc: 1.0000
Epoch 94/100
1/1 [===========================

In [19]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


# Loading Pretrained Word Embeddings

In [20]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Flatten

# from numpy import array
# from keras.preprocessing.text import one_hot
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers.embeddings import Embedding

In [21]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [22]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [23]:
"""In the last section, we used one_hot function to convert text to vectors. 
Another approach is to use Tokenizer function from keras.preprocessing.text library."""

'In the last section, we used one_hot function to convert text to vectors. \nAnother approach is to use Tokenizer function from keras.preprocessing.text library.'

In [24]:
from keras.preprocessing.text import Tokenizer

In [25]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [26]:
print(word_tokenizer.word_index)

{'movie': 1, 'it': 2, 'is': 3, 'the': 4, 'i': 5, 'was': 6, 'like': 7, 'a': 8, 'fantastic': 9, 'horrible': 10, 'acting': 11, 'pathetic': 12, 'not': 13, 'this': 14, 'an': 15, 'excellent': 16, 'move': 17, 'you': 18, 'should': 19, 'watch': 20, 'brilliant': 21, 'exceptionally': 22, 'good': 23, 'wonderfully': 24, 'directed': 25, 'and': 26, 'executed': 27, 'its': 28, 'series': 29, 'never': 30, 'watched': 31, 'such': 32, 'brillent': 33, 'wonderful': 34, 'waste': 35, 'of': 36, 'money': 37, 'picture': 38, 'very': 39, 'boring': 40, 'did': 41, 'will': 42, 'recommend': 43}


In [27]:
vocab_length = len(word_tokenizer.word_index) + 1

In [28]:
print(vocab_length)

44


In [29]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)


[[14, 3, 15, 16, 1], [4, 17, 6, 9, 5, 7, 2], [18, 19, 20, 2, 3, 21], [22, 23], [24, 25, 26, 27, 5, 7, 2], [28, 8, 9, 29], [30, 31, 32, 8, 33, 1], [2, 3, 8, 34, 1], [10, 11], [35, 36, 37], [12, 38], [2, 6, 39, 40], [5, 41, 13, 7, 4, 1], [4, 1, 6, 10], [5, 42, 13, 43], [4, 11, 3, 12]]


In [30]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(padded_sentences)

[[14  3 15 16  1  0  0]
 [ 4 17  6  9  5  7  2]
 [18 19 20  2  3 21  0]
 [22 23  0  0  0  0  0]
 [24 25 26 27  5  7  2]
 [28  8  9 29  0  0  0]
 [30 31 32  8 33  1  0]
 [ 2  3  8 34  1  0  0]
 [10 11  0  0  0  0  0]
 [35 36 37  0  0  0  0]
 [12 38  0  0  0  0  0]
 [ 2  6 39 40  0  0  0]
 [ 5 41 13  7  4  1  0]
 [ 4  1  6 10  0  0  0]
 [ 5 42 13 43  0  0  0]
 [ 4 11  3 12  0  0  0]]


In [31]:
print(word_count)

<function <lambda> at 0x00000111FCFD28B8>


In [32]:
longest_sentence

'The move was fantastic I like it'

In [33]:
length_long_sentence

7

In [34]:
length_long_sentence

7

In [35]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('\\Users\\HP\\glove.6B.50d.txt', encoding="utf8")

In [36]:
"""load the GloVe word embeddings and then create our embedding matrix that contains 
the words in our corpus and their corresponding values from GloVe embeddings"""

'load the GloVe word embeddings and then create our embedding matrix that contains \nthe words in our corpus and their corresponding values from GloVe embeddings'

In [37]:
for line in glove_file:
#     print(line)
    records = line.split()
    word = records[0]
    #print(word)
    vector_dimensions = asarray(records[1:], dtype='float32')
    #print(vector_dimensions)
    embeddings_dictionary [word] = vector_dimensions
    #print(embeddings_dictionary)

glove_file.close()

In [38]:
embedding_matrix = zeros((vocab_length, 50))
print(embedding_matrix.shape)
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
print(embedding_matrix)

(44, 50)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.30824     0.17223001 -0.23339    ... -0.98176998 -0.32146999
   0.99822998]
 [ 0.61183    -0.22071999 -0.10898    ... -0.043688   -0.097922
   0.16806   ]
 ...
 [ 0.042523   -0.21172     0.044739   ...  0.12102    -0.23999
   0.46656001]
 [ 0.81544     0.30171001  0.54720002 ...  0.31964999 -0.34740999
   0.41672   ]
 [ 0.52735001 -0.61475003 -0.42524001 ...  1.16630006 -0.10517
   1.20070004]]


In [39]:
vocab_length

44

In [40]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 50, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
#the output shape of embedding layer is (batch_size, input_length, output_dim),
#input_length argument (length of input sequences)
#model.add(Embedding(input_dim,output_dim, input_length=length_long_sentence))
#model.add(Embedding(50, 20, input_length=length_long_sentence))
#length_long_sentence: 7

In [41]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 50)             2200      
_________________________________________________________________
flatten_1 (Flatten)          (None, 350)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 351       
Total params: 2,551
Trainable params: 351
Non-trainable params: 2,200
_________________________________________________________________
None


In [42]:
vocab_length

44

In [43]:
44*50

2200

In [44]:
padded_sentences.shape, sentiments.shape,vocab_length,length_long_sentence

((16, 7), (16,), 44, 7)

In [45]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 0s/step - loss: 0.7128 - acc: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6987 - acc: 0.5625
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6849 - acc: 0.5625
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6713 - acc: 0.6250
Epoch 5/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6580 - acc: 0.6250
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6450 - acc: 0.6250
Epoch 7/100
1/1 [==============================] - 0s 0s/step - loss: 0.6322 - acc: 0.6250
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6197 - acc: 0.6250
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6075 - acc: 0.6875
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5955 - acc: 0.6875
Epoch 11/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5839 - 

1/1 [==============================] - 0s 2ms/step - loss: 0.1717 - acc: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1696 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 7ms/step - loss: 0.1676 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 999us/step - loss: 0.1656 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1637 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1618 - acc: 1.0000
Epoch 94/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1599 - acc: 1.0000
Epoch 95/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1581 - acc: 1.0000
Epoch 96/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1563 - acc: 1.0000
Epoch 97/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1545 - acc: 1.0000
Epoch 98/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1528 - 

In [46]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


# keras functional API

In [47]:
"""In the Keras Functional API, you have to define the input layer separately before the embedding layer.
In the input, layer you have to simply pass the length of input vector. 
To specify that previous layer as input to the next layer, 
the previous layer is passed as a parameter inside the parenthesis,
at the end of the next layer."""

'In the Keras Functional API, you have to define the input layer separately before the embedding layer.\nIn the input, layer you have to simply pass the length of input vector. \nTo specify that previous layer as input to the next layer, \nthe previous layer is passed as a parameter inside the parenthesis,\nat the end of the next layer.'

In [49]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [52]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
# trying to use Tensorflow v1.14 by installing it using pip.
deep_inputs = Input(shape=(length_long_sentence,))
embedding = Embedding(vocab_length, 50, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)(deep_inputs) # line A
flatten = Flatten()(embedding)
hidden = Dense(1, activation='sigmoid')(flatten)
model = Model(inputs=deep_inputs, outputs=hidden)

In [53]:
length_long_sentence

7

In [54]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 7, 50)             2200      
_________________________________________________________________
flatten_3 (Flatten)          (None, 350)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 351       
Total params: 2,551
Trainable params: 351
Non-trainable params: 2,200
_________________________________________________________________
None


In [55]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)

print('Accuracy: %f' % (accuracy*100))

Epoch 1/100
1/1 [==============================] - 0s 998us/step - loss: 0.6891 - acc: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 0s/step - loss: 0.6712 - acc: 0.6250
Epoch 3/100
1/1 [==============================] - 0s 0s/step - loss: 0.6540 - acc: 0.6875
Epoch 4/100
1/1 [==============================] - 0s 997us/step - loss: 0.6376 - acc: 0.6875
Epoch 5/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6220 - acc: 0.6250
Epoch 6/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6070 - acc: 0.6875
Epoch 7/100
1/1 [==============================] - 0s 984us/step - loss: 0.5927 - acc: 0.6875
Epoch 8/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5790 - acc: 0.6875
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5660 - acc: 0.6875
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 0.5535 - acc: 0.6875
Epoch 11/100
1/1 [==============================] - 0s 999us/step - loss: 0

1/1 [==============================] - 0s 0s/step - loss: 0.1606 - acc: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 995us/step - loss: 0.1588 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1569 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1552 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1534 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1517 - acc: 1.0000
Epoch 94/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1500 - acc: 1.0000
Epoch 95/100
1/1 [==============================] - 0s 1ms/step - loss: 0.1483 - acc: 1.0000
Epoch 96/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1467 - acc: 1.0000
Epoch 97/100
1/1 [==============================] - 0s 1ms/step - loss: 0.1451 - acc: 1.0000
Epoch 98/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1435 - a